# TEST THE CONNECTING FLOW FROM PYTHON SCRIPT TO UNITY CLOUD

In [1]:
import os
import requests
import datetime
import base64
from json import loads as json_loads,dumps

project_id = "dec49a5b-f225-4337-9ffb-3d095b81a994"
environment_id = "ee2d923d-2aab-451e-8d78-1fbdc487711a"
player_id = "oZFhVsAuYjXVfx1C6B2K8LMY4sN2"
SERVICE_ACCOUNT_CREDENTIALS = "Basic d2aedefb-5574-4234-9f21-9679914c6cd1:bxn_jkbj5SMF4ZtrljxdYiby2LJ_vJrZ"

In [2]:
def check_and_create_dir(dir_path):
  try:
    if not os.path.exists(dir_path):
      os.makedirs(dir_path)  # Create intermediate directories if needed
      print(f"Directory created: {dir_path}")
    else:
      print(f"Directory exists: {dir_path}")
    return True
  except OSError as e:
    print(f"Error creating directory: {e}")
    return False

In [3]:
current_dir = os.getcwd()
USER_MODEL_DIR = current_dir + '/user_models'
DATA_SAVE_DIR = current_dir + '/datasets'
check_and_create_dir(USER_MODEL_DIR)
check_and_create_dir(DATA_SAVE_DIR)

Directory exists: /Users/admin/Desktop/GameProjects/DataScience/Binhlai_Testing/user_models
Directory exists: /Users/admin/Desktop/GameProjects/DataScience/Binhlai_Testing/datasets


True

## 1. Connecting to Unity Cloud

### 1.1. Authenticate an API using service account credentials

In [4]:
url = f"https://services.api.unity.com/auth/v1/token-exchange"
method = "POST"
params = {"projectId":project_id, "environmentId":environment_id}
headers = {"Authorization": SERVICE_ACCOUNT_CREDENTIALS}
response1 = requests.request(method, url, headers=headers, params=params)
response1.text

'{"accessToken":"eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6InVuaXR5LWtleXM6MzU0OWFkNDMtN2RjYS00YTdkLTg2MWMtYjJmM2ZjZmMyZTAyIiwiamt1IjoiaHR0cHM6Ly9rZXlzLnNlcnZpY2VzLnVuaXR5LmNvbS8ifQ.eyJleHAiOjE3MTQ0NjAwNjAsImlhdCI6MTcxNDQ1NjQ2MCwibmJmIjoxNzE0NDU2NDYwLCJqdGkiOiJkNDAxM2E4NC0yOGIyLTQ2ZmEtOTA5OC05MzljODVkNzI5OTIiLCJzdWIiOiI1MGNhMmY0NS01ZjBmLTQ2ZjMtOGE1ZC0yMTUzNDcyNGYzODUiLCJ2ZXJzaW9uIjoxLCJpc3MiOiJodHRwczovL3NlcnZpY2VzLnVuaXR5LmNvbSIsImF1ZCI6WyJ1cGlkOmRlYzQ5YTViLWYyMjUtNDMzNy05ZmZiLTNkMDk1YjgxYTk5NCIsImVudklkOmVlMmQ5MjNkLTJhYWItNDUxZS04ZDc4LTFmYmRjNDg3NzExYSJdLCJzY29wZXMiOlsiYW1jLmFzc2V0X2xhYmVscy5hc3NpZ24iLCJhbWMuYXNzZXRfbGFiZWxzLnVuYXNzaWduIiwiYW1jLmFzc2V0cy5jcmVhdGUiLCJhbWMuYXNzZXRzLmRlbGV0ZSIsImFtYy5hc3NldHMuZG93bmxvYWQiLCJhbWMuYXNzZXRzLmxpc3QiLCJhbWMuYXNzZXRzLnB1Ymxpc2giLCJhbWMuYXNzZXRzLnJlYWQiLCJhbWMuYXNzZXRzLnN5bmMiLCJhbWMuYXNzZXRzLnRyYW5zZm9ybWF0aW9ucy5lbmQiLCJhbWMuYXNzZXRzLnRyYW5zZm9ybWF0aW9ucy5saXN0IiwiYW1jLmFzc2V0cy50cmFuc2Zvcm1hdGlvbnMucmVhZCIsImFtYy5hc3NldHMudHJhbnNmb3JtYXRp

In [5]:
data_response1 = response1.json()
access_token = data_response1["accessToken"]

### 1.2. Work with Cloud Save
Get another player's data

In [6]:
url = f"https://cloud-save.services.api.unity.com/v1/data/projects/{project_id}/players/{player_id}/items"
method = "GET"
headers = {"ProjectId": project_id,"Authorization":f"Bearer {access_token}"}
params = {"keys": ["MODELS"]}
response2 = requests.request(method, url, headers=headers, params=params)
response2.text

2024-04-19 11:02:35,364 urllib3.connectionpool [DEBUG] - Starting new HTTPS connection (1): cloud-save.services.api.unity.com:443
2024-04-19 11:02:35,605 urllib3.connectionpool [DEBUG] - https://cloud-save.services.api.unity.com:443 "GET /v1/data/projects/dec49a5b-f225-4337-9ffb-3d095b81a994/players/oZFhVsAuYjXVfx1C6B2K8LMY4sN2/items?keys=MODELS HTTP/1.1" 200 535


'{"results":[{"key":"MODELS","value":{"models":[{"accuracy":0,"features":["gross_profit_margin","profit_margin","trend_gross_margin","eps_on_mp"],"frequency":1,"index":0,"lastTrained":"3:55 PM","localDirectory":"./Assets/MAIN/AiAgent/Models/model_7a925f8b-83e6-4de3-832c-90b9d08ea210.txt","stability":0,"subscriptionId":"7a925f8b-83e6-4de3-832c-90b9d08ea210","trainedAmount":0}]},"writeLock":"56bb9c556ccb0a20a1cd84e88e6b5b83","modified":{"date":"2024-04-18T12:55:29Z"},"created":{"date":"2024-04-15T03:31:20Z"}}],"links":{"next":null}}'

## 2. Test loading an ML model from local files

### 2.1. Load an available model & save it as txt file

#### Load model configuration from Cloud Save

In [7]:
config_data = json_loads(response2.text)
models_data = config_data['results'][0]['value']['models']
model_config = models_data[0]
model_config

{'accuracy': 0,
 'features': ['gross_profit_margin',
  'profit_margin',
  'trend_gross_margin',
  'eps_on_mp'],
 'frequency': 1,
 'index': 0,
 'lastTrained': '3:55 PM',
 'localDirectory': './Assets/MAIN/AiAgent/Models/model_7a925f8b-83e6-4de3-832c-90b9d08ea210.txt',
 'stability': 0,
 'subscriptionId': '7a925f8b-83e6-4de3-832c-90b9d08ea210',
 'trainedAmount': 0}

##### Check if the user has the model or not. 
- If not, generate a new one with model's index and locate it in a folder named following player_Id.
- Load the model from storage.

In [8]:
observation_size = len(model_config['features'])
model_index = model_config['index']

## 3. Work with UGC

#### 3.1. Take a look on current contents

Get details about a content item

In [2]:
url = f"https://services.api.unity.com/ugc/v1/projects/{project_id}/environments/{environment_id}/content/search"
method = "GET"
headers = {"Authorization": SERVICE_ACCOUNT_CREDENTIALS}
# params = {"filters": ["visibility,eq,private"]}
response5 = requests.request(method, url, headers=headers)#,params=params)
response5.text

ERROR! Session/line number was not unique in database. History logging moved to new session 4


'{"offset":0,"limit":25,"total":0,"results":[{"id":"f769ffb4-7e50-4760-9984-409c88b1732a","name":"Booster","customId":null,"description":"A description","visibility":"public","moderationStatus":"approved","version":"ebb38503-46da-469d-8570-35997147ef1e","createdAt":"2024-04-08T16:19:25.424963Z","updatedAt":"2024-04-18T06:06:53.985253Z","deletedAt":null,"projectId":"dec49a5b-f225-4337-9ffb-3d095b81a994","environmentId":"ee2d923d-2aab-451e-8d78-1fbdc487711a","creatorAccountId":"1EYlvVKJ4Y59pt0JaR2BoAffdJ8m","thumbnailUrl":"https://ugc-prd.unity3d.com/ugc-prd/projects/dec49a5b-f225-4337-9ffb-3d095b81a994/envs/ee2d923d-2aab-451e-8d78-1fbdc487711a/contents/f769ffb4-7e50-4760-9984-409c88b1732a/ebb38503-46da-469d-8570-35997147ef1e_t?TOKEN=exp=1714458748~acl=/ugc-prd/projects/dec49a5b-f225-4337-9ffb-3d095b81a994/envs/ee2d923d-2aab-451e-8d78-1fbdc487711a/contents/f769ffb4-7e50-4760-9984-409c88b1732a/*~hmac=f4333fd5c35613f148ff0b88cfe97d847f7e1c20579cd2c4bcfd8a695ceca22e","downloadUrl":"https://

#### 3.2. Try to create new content

Create a dummy account for UGC posting and set this account as a moderator

In [3]:
username = "BinhLai"
password = "Imz@16188"

In [83]:
# url = "https://player-auth.services.api.unity.com/v1/authentication/usernamepassword/sign-up"
# method = "POST"
# headers = {"ProjectId": project_id,"UnityEnvironment":"dev"}
# json = {"username":username,"password":password}
# response8 = requests.request(method, url, headers=headers,json=json)
# response8.text

Sign in the dummpy account

In [4]:
url = "https://player-auth.services.api.unity.com/v1/authentication/usernamepassword/sign-in"
method = "POST"
headers = {"ProjectId": project_id,"UnityEnvironment":"dev"}
json = {"username":username,"password":password}
response9 = requests.request(method, url, headers=headers,json=json)
response9.text

'{"expiresIn":3599,"idToken":"eyJhbGciOiJSUzI1NiIsImtpZCI6InB1YmxpYzo2NzQ2QjA5NC0zODNCLTRFMDYtQjA0OS04OUU4MTU1NjdBOUQiLCJ0eXAiOiJKV1QifQ.eyJhdWQiOlsiaWRkOjUzMzJjYmM3LTM5ODQtNGRlYi04NzIxLWFjNDhhZGQ0NWU0ZiIsImVudk5hbWU6ZGV2IiwiZW52SWQ6ZWUyZDkyM2QtMmFhYi00NTFlLThkNzgtMWZiZGM0ODc3MTFhIiwidXBpZDpkZWM0OWE1Yi1mMjI1LTQzMzctOWZmYi0zZDA5NWI4MWE5OTQiXSwiZXhwIjoxNzE0NDYyMDU2LCJpYXQiOjE3MTQ0NTg0NTYsImlkZCI6IjUzMzJjYmM3LTM5ODQtNGRlYi04NzIxLWFjNDhhZGQ0NWU0ZiIsImlzcyI6Imh0dHBzOi8vcGxheWVyLWF1dGguc2VydmljZXMuYXBpLnVuaXR5LmNvbSIsImp0aSI6IjBkZGYwODlmLTdjMjktNDhhNi05NzUzLWM1OWFmYTQ1ZGEyNCIsIm5iZiI6MTcxNDQ1ODQ1NiwicHJvamVjdF9pZCI6ImRlYzQ5YTViLWYyMjUtNDMzNy05ZmZiLTNkMDk1YjgxYTk5NCIsInNpZ25faW5fcHJvdmlkZXIiOiJ1c2VybmFtZXBhc3N3b3JkIiwic3ViIjoiejI2VWIzdGpMbjJUb0JqTU85RnF5MW1WUFlKcSIsInRva2VuX3R5cGUiOiJhdXRoZW50aWNhdGlvbiIsInZlcnNpb24iOiIxIn0.NAy4wZzHUD8Wl38xvqvcFcR5vNmc4doYbyqGL-AIushnoN9THt-7WgPuY0_dZwBmH7baPsBxwrfODvfUL3CtvOwTgu3C_ajkz47GfBRcu4PAnI5bTf0SyoRrTWizvx4S5zOnlFCqywB7nRCRiQx5Tnl1mXRjJ-m8t-nWZvOe6CP

In [5]:
session_token = json_loads(response9.text)
session_token = session_token['idToken']

Try to post player's content

In [10]:
# url = f"https://ugc.services.api.unity.com/v1/projects/{project_id}/environments/{environment_id}/content"
# method = "POST"
# headers = {"Authorization": f"Bearer {session_token}"}
# json = {'name': "Admin_Add_Content", "description": "Test Admin add content", "visibility": "public"}
# response10 = requests.request(method, url, headers=headers, json=json)
# response10.text

In [10]:
# upload_data = json_loads(response8.text)
# signedUrl = upload_data["uploadContentUrl"]
# httpMethod = "PUT"
# requiredHeaders = upload_data["uploadContentHeaders"]
# headers = {key: value[0] for key, value in requiredHeaders.items()}

# data = content
# response11 = requests.request(url=signedUrl, method=httpMethod, headers=headers, data=data)
# response11.text

#### 3.3. Content subscription

Check if a certain piece of content is subscribed by the current user

In [111]:
# content_id = upload_data['content']['id']
# url = f"https://ugc.services.api.unity.com/v1/subscriptions/projects/{project_id}/environments/{environment_id}/content/{content_id}"
# method = "POST"
# headers = {"Authorization": f"Bearer {session_token}"}
# response12 = requests.request(method, url, headers=headers)
# response12.text

2024-04-18 10:15:17,405 urllib3.connectionpool [DEBUG] - Starting new HTTPS connection (1): ugc.services.api.unity.com:443
2024-04-18 10:15:17,629 urllib3.connectionpool [DEBUG] - https://ugc.services.api.unity.com:443 "POST /v1/subscriptions/projects/dec49a5b-f225-4337-9ffb-3d095b81a994/environments/ee2d923d-2aab-451e-8d78-1fbdc487711a/content/7a925f8b-83e6-4de3-832c-90b9d08ea210 HTTP/1.1" 404 189


'{"status":404,"title":"Not Found","type":"https://services.docs.unity.com/docs/errors/#54","requestId":"1b77b2aa-1b6a-4558-abcf-0279b45a27f3","detail":"Object could not be found","code":54}'

Try get a content by it's Id

In [114]:
url = f"https://services.api.unity.com/ugc/v1/projects/{project_id}/environments/{environment_id}/content/{content_id}"
method = "GET"
headers = {"Authorization": SERVICE_ACCOUNT_CREDENTIALS}
response13 = requests.request(method, url, headers=headers)
response13.text

2024-04-18 10:15:48,076 urllib3.connectionpool [DEBUG] - Starting new HTTPS connection (1): services.api.unity.com:443
2024-04-18 10:15:48,371 urllib3.connectionpool [DEBUG] - https://services.api.unity.com:443 "GET /ugc/v1/projects/dec49a5b-f225-4337-9ffb-3d095b81a994/environments/ee2d923d-2aab-451e-8d78-1fbdc487711a/content/7a925f8b-83e6-4de3-832c-90b9d08ea210 HTTP/1.1" 200 None


'{"id":"7a925f8b-83e6-4de3-832c-90b9d08ea210","name":"Admin_Add_Content","customId":null,"description":"Test Admin add content","visibility":"public","moderationStatus":"approved","version":"6472626a-8a38-4672-b7e9-334437b59af8","createdAt":"2024-04-18T06:25:20.804439Z","updatedAt":"2024-04-18T06:39:36.893817Z","deletedAt":null,"projectId":"dec49a5b-f225-4337-9ffb-3d095b81a994","environmentId":"ee2d923d-2aab-451e-8d78-1fbdc487711a","creatorAccountId":"z26Ub3tjLn2ToBjMO9Fqy1mVPYJq","thumbnailUrl":"https://ugc-prd.unity3d.com/ugc-prd/projects/dec49a5b-f225-4337-9ffb-3d095b81a994/envs/ee2d923d-2aab-451e-8d78-1fbdc487711a/contents/7a925f8b-83e6-4de3-832c-90b9d08ea210/6472626a-8a38-4672-b7e9-334437b59af8_t?TOKEN=exp=1713424819~acl=/ugc-prd/projects/dec49a5b-f225-4337-9ffb-3d095b81a994/envs/ee2d923d-2aab-451e-8d78-1fbdc487711a/contents/7a925f8b-83e6-4de3-832c-90b9d08ea210/*~hmac=983b00e5d7c2812cdc5f5a950f30567cbebb79a71875f0bd7d59e3c1e88de8d3","downloadUrl":"https://ugc-prd.unity3d.com/ugc-p

In [115]:
uploaded_content_id = json_loads(response13.text)
print(uploaded_content_id['Details'])

KeyError: 'Details'

Update contentId in player's CloudSave

In [131]:
model_config['subscriptionId'] = content_id

[{'features': ['gross_profit_margin',
   'profit_margin',
   'trend_gross_margin',
   'eps_on_mp'],
  'frequency': 1,
  'index': 0,
  'lastTrained': '2024-04-16T00:00:00+03:00',
  'trainedAmount': 0,
  'subscriptionId': '7a925f8b-83e6-4de3-832c-90b9d08ea210'}]

In [31]:
url = f"https://services.api.unity.com/cloud-save/v1/data/projects/{project_id}/environments/{environment_id}/players/{player_id}/items"
method = "POST"
headers = {"Authorization": SERVICE_ACCOUNT_CREDENTIALS}
json = {"key": "MODELS", "value": config_data['results'][0]['value']}
response14 = requests.request(method, url, headers=headers, json=json)
response14.text

2024-04-18 11:36:01,256 urllib3.connectionpool [DEBUG] - Starting new HTTPS connection (1): services.api.unity.com:443
2024-04-18 11:36:01,611 urllib3.connectionpool [DEBUG] - https://services.api.unity.com:443 "POST /cloud-save/v1/data/projects/dec49a5b-f225-4337-9ffb-3d095b81a994/environments/ee2d923d-2aab-451e-8d78-1fbdc487711a/players/oZFhVsAuYjXVfx1C6B2K8LMY4sN2/items HTTP/1.1" 200 48


'{"writeLock":"2357c4cc1d01f8bcb85e264f807f4ee7"}'

## 4. Modify UGC's content

Create a dummy account for UGC posting and set this account as a moderator

In [11]:
# username = "BinhLai"
# password = "Imz@16188"

In [ ]:
# url = "https://player-auth.services.api.unity.com/v1/authentication/usernamepassword/sign-up"
# method = "POST"
# headers = {"ProjectId": project_id,"UnityEnvironment":"dev"}
# json = {"username":username,"password":password}
# response8 = requests.request(method, url, headers=headers,json=json)
# response8.text

Sign in the dummpy account

In [ ]:
# url = "https://player-auth.services.api.unity.com/v1/authentication/usernamepassword/sign-in"
# method = "POST"
# headers = {"ProjectId": project_id,"UnityEnvironment":"dev"}
# json = {"username":username,"password":password}
# response9 = requests.request(method, url, headers=headers,json=json)
# response9.text

In [ ]:
# session_token = json_loads(response7.text)
# session_token = session_token['idToken']

Get player's contents

In [ ]:
# url = f"https://ugc.services.api.unity.com/v1/projects/{project_id}/environments/{environment_id}/content"
# method = "POST"
# headers = {"Authorization": f"Bearer {session_token}"}
# json = {'name': "Admin_Add_Content", "description": "Test Admin add content", "visibility": "public"}
# response10 = requests.request(method, url, headers=headers, json=json)
# response10.text

Delete contents

In [ ]:
# for content in contents['results']:
#     url = f"https://services.api.unity.com/ugc/v1/projects/{project_id}/environments/{environment_id}/content/{content['id']}"
#     method = "DELETE"
#     headers = {"Authorization": SERVICE_ACCOUNT_CREDENTIALS}
#     response6 = requests.request(method, url, headers=headers)

Restore content

In [ ]:
# for content in contents['results']:
#     url = f"https://services.api.unity.com/ugc/v1/projects/{project_id}/environments/{environment_id}/content/{content['id']}/restore"
#     method = "POST"
#     headers = {"Authorization": SERVICE_ACCOUNT_CREDENTIALS}
#     response7 = requests.request(method, url, headers=headers)

Get content by Id

In [39]:
content_id = 'd30ec280-57a2-43da-b675-cb460fbf4821'
url = f"https://services.api.unity.com/ugc/v1/projects/{project_id}/environments/{environment_id}/content/{content_id}"
method = "GET"
headers = {"Authorization": SERVICE_ACCOUNT_CREDENTIALS}
response11 = requests.request(method, url, headers=headers)
response11.text

'{"id":"d30ec280-57a2-43da-b675-cb460fbf4821","name":"oZFhVsAuYjXVfx1C6B2K8LMY4sN2__12_features","customId":null,"description":"Test create new model","visibility":"public","moderationStatus":"approved","version":"f78562ec-7449-4fc6-89cb-3b17722a0773","createdAt":"2024-04-29T12:49:26.740584Z","updatedAt":"2024-04-30T06:39:17.453128Z","deletedAt":null,"projectId":"dec49a5b-f225-4337-9ffb-3d095b81a994","environmentId":"ee2d923d-2aab-451e-8d78-1fbdc487711a","creatorAccountId":"z26Ub3tjLn2ToBjMO9Fqy1mVPYJq","thumbnailUrl":"https://ugc-prd.unity3d.com/ugc-prd/projects/dec49a5b-f225-4337-9ffb-3d095b81a994/envs/ee2d923d-2aab-451e-8d78-1fbdc487711a/contents/d30ec280-57a2-43da-b675-cb460fbf4821/f78562ec-7449-4fc6-89cb-3b17722a0773_t?TOKEN=exp=1714459747~acl=/ugc-prd/projects/dec49a5b-f225-4337-9ffb-3d095b81a994/envs/ee2d923d-2aab-451e-8d78-1fbdc487711a/contents/d30ec280-57a2-43da-b675-cb460fbf4821/*~hmac=871b3daaba6218b17594d51d113f5f0aff5ec40ad3403a4e0ab7219862f21c4a","downloadUrl":"https://ug

Retrieve all tagId

In [59]:
url = f"https://services.api.unity.com/ugc/v1/projects/{project_id}/environments/{environment_id}/tags"
method = "GET"
headers = {"Authorization": SERVICE_ACCOUNT_CREDENTIALS}
response12 = requests.request(method, url, headers=headers)
response12.text

'[{"id":"d78741ef-788b-4a6c-8351-8db5e22df468","name":"gross_profit_margin","projectId":"dec49a5b-f225-4337-9ffb-3d095b81a994","environmentId":"ee2d923d-2aab-451e-8d78-1fbdc487711a"},{"id":"ef3fc361-7e0e-47af-b030-2d6f350d9fcf","name":"trend_profit_margin","projectId":"dec49a5b-f225-4337-9ffb-3d095b81a994","environmentId":"ee2d923d-2aab-451e-8d78-1fbdc487711a"},{"id":"8f3638e6-c18e-42c8-a649-f31c15916d08","name":"liabilities_on_equity","projectId":"dec49a5b-f225-4337-9ffb-3d095b81a994","environmentId":"ee2d923d-2aab-451e-8d78-1fbdc487711a"},{"id":"3663ac1a-9cfb-49d9-a9dc-f4758338cc32","name":"dividend_on_mp","projectId":"dec49a5b-f225-4337-9ffb-3d095b81a994","environmentId":"ee2d923d-2aab-451e-8d78-1fbdc487711a"},{"id":"e9c9ee8e-0efe-4652-bdae-abc643e83a38","name":"material","projectId":"dec49a5b-f225-4337-9ffb-3d095b81a994","environmentId":"ee2d923d-2aab-451e-8d78-1fbdc487711a"},{"id":"b2f4be22-59bb-44d2-9f43-d4c7fd1b7390","name":"sga_ratio","projectId":"dec49a5b-f225-4337-9ffb-3d095b

In [10]:
tags = json_loads(response12.text)

Retrieve a specific content based on a certain tagId

In [65]:
url = f"https://services.api.unity.com/ugc/v1/projects/{project_id}/environments/{environment_id}/content/search"
method = "GET"
headers = {"Authorization": SERVICE_ACCOUNT_CREDENTIALS}
# json = {'name': "Admin_Add_Content", "description": "Test Admin add content", "visibility": "public"}
params = {'tags': selected_tags}
response10 = requests.request(method, url, headers=headers, params=params)
print(response10.url)
print(response10.text)

https://services.api.unity.com/ugc/v1/projects/dec49a5b-f225-4337-9ffb-3d095b81a994/environments/ee2d923d-2aab-451e-8d78-1fbdc487711a/content/search?tags=d78741ef-788b-4a6c-8351-8db5e22df468&tags=3663ac1a-9cfb-49d9-a9dc-f4758338cc32&tags=b2f4be22-59bb-44d2-9f43-d4c7fd1b7390&tags=2e9244ec-407a-41c4-ad8e-0364eccabdc5&tags=a4a941bf-4e0b-443b-85c3-a259160cdb9b&tags=be5b3180-2950-463d-b0ae-1d6d3998331b&tags=c87b782f-5f11-483c-b7fa-ce31caf9d11f&tags=195ce02b-1e20-4c80-973d-586d23a7d4bf
{"offset":0,"limit":25,"total":0,"results":[{"id":"d30ec280-57a2-43da-b675-cb460fbf4821","name":"oZFhVsAuYjXVfx1C6B2K8LMY4sN2__12_features","customId":null,"description":"Test create new model","visibility":"public","moderationStatus":"approved","version":"f78562ec-7449-4fc6-89cb-3b17722a0773","createdAt":"2024-04-29T12:49:26.740584Z","updatedAt":"2024-04-30T06:58:51.042414Z","deletedAt":null,"projectId":"dec49a5b-f225-4337-9ffb-3d095b81a994","environmentId":"ee2d923d-2aab-451e-8d78-1fbdc487711a","creatorAccou

Update content's tags

In [52]:
selected_tags = [x['id'] for x in tags if (x['name'] in ['model','dep_ratio','dividend_on_mp','ebit_on_int','eps_on_mp',
                                                  'gross_profit_margin','profit_margin','sga_ratio'])]
selected_tags

['d78741ef-788b-4a6c-8351-8db5e22df468',
 '3663ac1a-9cfb-49d9-a9dc-f4758338cc32',
 'b2f4be22-59bb-44d2-9f43-d4c7fd1b7390',
 '2e9244ec-407a-41c4-ad8e-0364eccabdc5',
 'a4a941bf-4e0b-443b-85c3-a259160cdb9b',
 'be5b3180-2950-463d-b0ae-1d6d3998331b',
 'c87b782f-5f11-483c-b7fa-ce31caf9d11f',
 '195ce02b-1e20-4c80-973d-586d23a7d4bf']

In [53]:
updated_content = json_loads(response11.text)
updated_content_name = updated_content['name']
updated_content_des = updated_content['description']
updated_content_metadata = updated_content['metadata']

In [54]:
url = f"https://services.api.unity.com/ugc/v1/projects/{project_id}/environments/{environment_id}/content/{content_id}/details"
method = "PUT"
headers = {"Authorization": SERVICE_ACCOUNT_CREDENTIALS}
json = {'name': updated_content_name, "description": updated_content_des,'metadata': updated_content_metadata, 'tagsId': selected_tags}
response13 = requests.request(method, url, headers=headers, json=json)
response13.text

'{"id":"d30ec280-57a2-43da-b675-cb460fbf4821","name":"oZFhVsAuYjXVfx1C6B2K8LMY4sN2__12_features","customId":null,"description":"Test create new model","visibility":"public","moderationStatus":"approved","version":"f78562ec-7449-4fc6-89cb-3b17722a0773","createdAt":"2024-04-29T12:49:26.740584Z","updatedAt":"2024-04-30T06:58:51.0424148Z","deletedAt":null,"projectId":"dec49a5b-f225-4337-9ffb-3d095b81a994","environmentId":"ee2d923d-2aab-451e-8d78-1fbdc487711a","creatorAccountId":"z26Ub3tjLn2ToBjMO9Fqy1mVPYJq","thumbnailUrl":"https://ugc-prd.unity3d.com/ugc-prd/projects/dec49a5b-f225-4337-9ffb-3d095b81a994/envs/ee2d923d-2aab-451e-8d78-1fbdc487711a/contents/d30ec280-57a2-43da-b675-cb460fbf4821/f78562ec-7449-4fc6-89cb-3b17722a0773_t?TOKEN=exp=1714460578~acl=/ugc-prd/projects/dec49a5b-f225-4337-9ffb-3d095b81a994/envs/ee2d923d-2aab-451e-8d78-1fbdc487711a/contents/d30ec280-57a2-43da-b675-cb460fbf4821/*~hmac=f0e53970d949615a41939b2e675ad3234adc1d9a5cfe7c68a55909bf392f0cb8","downloadUrl":"https://u

Update content's metadata (as group's inforamtion)

In [16]:
# selected_tag = [x for x in tags if x['name'] == 'material'][0]
# selected_tag['id']
# metadata = dumps({'accuracy': 1.0, 'stability': 0.5, 'winTransaction': 3})
# metadata

In [17]:
# contents = json_loads(response10.text)['results']

# for content in contents:
#     content_id = content['id']
#     url = f"https://services.api.unity.com/ugc/v1/projects/{project_id}/environments/{environment_id}/content/{content_id}/details"
#     method = "PUT"
#     headers = {"Authorization": SERVICE_ACCOUNT_CREDENTIALS}
#     json = {'name': contents[0]['name'], "description": contents[0]['description'],'metadata': metadata, 'tagsId': [selected_tag['id']]}
#     response14 = requests.request(method, url, headers=headers, json=json)

# response14.text

In [52]:
json

{'name': 'oZFhVsAuYjXVfx1C6B2K8LMY4sN2_material_0',
 'description': 'A another new description!',
 'metadata': {'accuracy': 1.0, 'stability': 0.5, 'winTransaction': 3},
 'tagsId': ['e9c9ee8e-0efe-4652-bdae-abc643e83a38']}

In [10]:
contents = json_loads(response10.text)['results']

Update user's content as an Admin

In [37]:
url = f"https://services.api.unity.com/ugc/v1/projects/{project_id}/environments/{environment_id}/content/{contents[0]['id']}/details"
method = "PUT"
headers = {"Authorization": SERVICE_ACCOUNT_CREDENTIALS}
json = {'name': contents[0]['name'], "description": contents[0]['description'],'metadata': metadata, 'tagsId': [selected_tag['id']]}
response15 = requests.request(method, url, headers=headers, json=json)
response15.text

'{"title":"Invalid input passed to API. Fields: metadata,metadata","detail":"Unexpected character encountered while parsing value: {. Path \'metadata\', line 1, position 105.","details":["Unexpected character encountered while parsing value: {. Path \'metadata\', line 1, position 105.","After parsing a value an unexpected character was encountered: :. Path \'metadata\', line 1, position 115."],"status":22020}'